<a href="https://colab.research.google.com/github/kaluznys/uczenie_maszynowe_UW/blob/main/praca_domowa6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework Assignment: Understanding Splitting Criteria in CART for Regression**
---------------------

In this assignment, you will explore three common formulations of the splitting criterion used in **CART (Classification and Regression Trees)** for **regression problems**:

1. **Local RSS Minimization**  
2. **RSS Gain Maximization**  
3. **Total RSS Minimization**

You will investigate whether any of these criteria are equivalent, and you will design an experiment to determine which criterion is actually employed in a standard implementation such as **scikit-learn’s DecisionTreeRegressor**.



## **The Problem**

Many treatments of CART for regression describe the split selection process in different ways. Below are three frequently cited formulations. Suppose we have a dataset with features $X$ and target $y$, and we seek to choose a feature $X_j$ and a threshold $t$ to split the data into two regions $R_1(X_j, t)$ and $R_2(X_j, t)$. Denote by $\bar{y}_{R_m}$ the mean of targets within region $R_m$.

1. **Local RSS Minimization**  
   We select the feature and threshold that minimize the **sum of squared errors** in the two resulting child nodes:
   $$
   (X_j^*, t^*) = \arg\min_{X_j, t} \sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2.
   $$

2. **RSS Gain Maximization**  

   It is also a local method, looking only at a parent and two child nodes.

   We select the feature and threshold that maximize the **reduction** in RSS, computed by subtracting the RSS of the two child nodes from the RSS in the parent node:
   $$
   (X_j^*, t^*) = \arg\max_{X_j, t} \Bigl\{
   \underbrace{\sum_{i : x_i \in \text{Parent}} (y_i - \bar{y})^2}_{\text{Parent RSS}}
   \;-\;
   \underbrace{\sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2}_{\text{Children RSS}}
   \Bigr\}.
   $$

3. **Total RSS Minimization**  
   For a dataset $\{(x_i, y_i)\}_{i=1}^N$ with features $X$ and target $y$, let $T$ be the current tree.

   For any split on feature $X_j$ at threshold $t$, define $T(X_j, t)$ as the new tree obtained by splitting one leaf of $T$ into two leaves $R_1(X_j, t)$ and $R_2(X_j, t)$.
   
   Let $\mathrm{Leaves}(T(X_j, t))$ be the set of all leaf indices in this new tree. For each leaf $m \in \mathrm{Leaves}(T(X_j, t))$, define:
   $$
   R_m = \{\, i \,\mid\, x_i \text{ ends in leaf } m\}.
   $$

   $R_m$ set collects all data indices $i$ whose feature vector $x_i$ is classified into the leaf node $m$ when passed through the tree $T(X_j,t)$. In other words, each leaf node $m$ in $T(X_j, t)$ corresponds to a unique path of splits, and any data point $x_i$ that follows that path is assigned to the leaf $m$; hence, it belongs to $R_m$.

   $R_m$ sets for all leafs $m \in \mathrm{Leaves}(T(X_j, t))$ define a partition of all indices.

   Then the objective of **minimizing total Residual Sum of Squares (total RSS)** is stated as:
   $$
   (X_j^*, t^*) = \arg\min_{(X_j, t)} \sum_{m \in \mathrm{Leaves}(T(X_j, t))}
   \sum_{i \in R_m} \Bigl(y_i - \overline{y}_{R_m}\Bigr)^2,
   $$
   where
   $$
   \overline{y}_{R_m} = \frac{1}{\lvert R_m \rvert}
   \sum_{i \in R_m} y_i
   $$
   is the mean response in leaf $m$.


## **Research Questions**

1. **Equivalence Analysis**  
   Determine whether the above formulations are equivalent or if they can yield different split choices. Specifically:
   - Are *local RSS minimization* and *RSS gain maximization* equivalent?
   - Does *total RSS minimization* coincide with either of these two, or is it distinct?
   
2. **Empirical Experiment**  
   Design and conduct a Python experiment to determine which of these formulations is implemented in `scikit-learn` in `DecisionTreeRegressor`. Present numerical results and plots to support your conclusion.


## **Tasks & Deliverables**

1. **Formulation Analysis**  
   - Compare *local RSS minimization*, *RSS gain maximization*, and *total RSS minimization*.
   - If you find that any pair of formulations is equivalent, provide a concise proof.  
   - If you find that they differ, construct a counterexample.

2. **Empirical Verification**  
   - Create a small artificial dataset and train a `DecisionTreeRegressor` from `scikit-learn`.
   - The dataset must be designed in a way that uniquely identifies the formulation used. Provide a short code snippet and a plot or table to support your conclusion.

3. **Report**  
   - Summarize your theoretical insights and empirical findings in a **Colab notebook**.
   - Include the relevant proofs, code, discussion, and conclusions.
   - Place the notebook in your **GitHub repository** for this course, add a link to it in your README.md and add an **“Open in Colab”** badge in the notebook so it can be launched directly.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
np.random.seed(42)
n_samples=50
ages = np.random.randint(20, 70, size=n_samples)
incomes = np.random.normal(loc=60000, scale=15000, size=n_samples).astype(int)
house_sizes = np.random.normal(loc=50, scale=15, size=n_samples).astype(int)
nr_of_houses = np.random.poisson(lam=1, size=n_samples)
nr_of_houses  = np.ceil(nr_of_houses)+1
locations = np.random.choice(['urban', 'suburban', 'rural'], size=n_samples)

In [ ]:
df = pd.DataFrame({
    'Age': ages,
    'Income': incomes,
    'HouseSize': house_sizes,
    'NrOfHouses': nr_of_houses,
    'Location': locations,
})
df = pd.get_dummies(df, "location")

In [ ]:
house_prices = (
    50000 +
    (ages * -100) +
    (incomes * 0.5) +
    (house_sizes * 4800) +
    (nr_of_houses * 10000) +
    (df.location_rural*-50000) +
    (df.location_suburban*30000)+
    (df.location_urban*-30000) +
    np.random.normal(0, 30000, size=n_samples)
)
df['price'] = house_prices
df.head()

,Age,Income,HouseSize,NrOfHouses,location_rural,location_suburban,location_urban,price
0,58,64365,41,2.0,True,False,False,289993.904234
1,48,50466,27,2.0,False,True,False,253889.530002
2,34,44676,38,1.0,False,True,False,307825.290352
3,62,57573,61,1.0,False,True,False,367260.345794
4,27,51995,46,3.0,False,False,True,365768.578722


In [ ]:
# liczy rss
def rss(y):
    if len(y) == 0:
        return 0
    return np.sum((y - np.mean(y)) ** 2)

In [ ]:
# definicja klasy węzła z atrybutami definiującymi podział (i warunek go indukujący)
class TreeNode:
    def __init__(self, feature=None, value=None, left=None, right=None, prediction=None):
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
        self.prediction = prediction

In [ ]:
# dla danego węzła zwraca dane do warunku na najlepszy podział wzgl. rss gain
def best_split_rss_gain(X, y):
    best_feature, best_value = None, None
    best_rss = float('-inf')

    for feature_idx in range(X.shape[1]):
        values = np.unique(X[:, feature_idx])
        for val in values:
          left_mask = X[:, feature_idx] <= val
          right_mask = ~left_mask
          if left_mask.sum() == 0 or right_mask.sum() == 0:
              continue

          y_left, y_right = y[left_mask], y[right_mask]

          rss_gain = rss(y)-(rss(y_left) + rss(y_right))  # rss gain

          if rss_gain > best_rss:
              best_rss = rss_gain
              best_feature = feature_idx
              best_value = val

    return best_feature, best_value, best_rss

In [ ]:
# to samo co wyżej, ale dla local rss
def best_split_rss_local(X, y):
    best_feature, best_value = None, None
    best_rss = float('inf')

    for feature_idx in range(X.shape[1]):
        values = np.unique(X[:, feature_idx])
        for val in values:
          left_mask = X[:, feature_idx] <= val
          right_mask = ~left_mask
          if left_mask.sum() == 0 or right_mask.sum() == 0:
              continue

          y_left, y_right = y[left_mask], y[right_mask]

          rss_gain = rss(y_left) + rss(y_right)  # Local RSS

          if rss_gain < best_rss:
              best_rss = rss_gain
              best_feature = feature_idx
              best_value = val

    return best_feature, best_value, best_rss

In [ ]:
# Liczy total rss rekurencyjnie na podstawie zbioru w rodzicu i danych podziału
def compute_total_rss(X, y, node):
    if node.prediction is not None:
        return rss(y)

    left_mask = X[:, node.feature] <= node.value
    right_mask = ~left_mask

    rss_left = compute_total_rss(X[left_mask], y[left_mask], node.left)
    rss_right = compute_total_rss(X[right_mask], y[right_mask], node.right)

    return rss_left + rss_right

In [ ]:
#  rekurencyjna budowa drzewa na podstawie podziałów wyznaczonych przez best_split...
def build_tree_rss_gain(X, y, max_depth=3, depth=0):
    if depth >= max_depth:
        return TreeNode(prediction=np.mean(y))

    feature, value, split_rss_gain = best_split_rss_gain(X, y)

    if feature is None:
        return TreeNode(prediction=np.mean(y))

    left_mask = X[:, feature] <= value
    right_mask = ~left_mask

    left = build_tree_rss_gain(X[left_mask], y[left_mask], max_depth, depth + 1)
    right = build_tree_rss_gain(X[right_mask], y[right_mask], max_depth, depth + 1)

    return TreeNode(feature=feature, value=value, left=left, right=right)

In [ ]:
# to samo co wyżej dla rss local
def build_tree_rss_local(X, y, max_depth=3, depth=0):
    if depth >= max_depth:
        return TreeNode(prediction=np.mean(y))

    feature, value, split_rss_gain = best_split_rss_local(X, y)

    if feature is None:
        return TreeNode(prediction=np.mean(y))

    left_mask = X[:, feature] <= value
    right_mask = ~left_mask

    left = build_tree_rss_local(X[left_mask], y[left_mask], max_depth, depth + 1)
    right = build_tree_rss_local(X[right_mask], y[right_mask], max_depth, depth + 1)

    return TreeNode(feature=feature, value=value, left=left, right=right)

In [ ]:
def build_tree_total_rss(X, y, max_depth=2, depth=0):
    if depth >= max_depth:
        return TreeNode(prediction=np.mean(y))

    best_tree = None
    best_rss_total = float('inf')

    for feature_idx in range(X.shape[1]):
        values = np.unique(X[:, feature_idx])
        for val in values:
            left_mask = X[:, feature_idx] <= val
            right_mask = ~left_mask

            y_left, y_right = y[left_mask], y[right_mask]
            X_left, X_right = X[left_mask], X[right_mask]

            # pomijamy puste podziały, gdy val jest największą wartością
            if len(y_left) == 0 or len(y_right) == 0:
                continue

            # rekurencyjnie budujemy
            left_tree = build_tree_total_rss(X_left, y_left, max_depth, depth + 1)
            right_tree = build_tree_total_rss(X_right, y_right, max_depth, depth + 1)

            # odwołanie do funkcji liczącej rss w celu doboru optimum
            rss_left = compute_total_rss(X_left, y_left, left_tree)
            rss_right = compute_total_rss(X_right, y_right, right_tree)
            rss_total = rss_left + rss_right

            if rss_total < best_rss_total:
                best_rss_total = rss_total
                best_tree = TreeNode(
                    feature=feature_idx,
                    value=val,
                    left=left_tree,
                    right=right_tree
                )

    # jeśli nie opłaca się w ogóle dzielić
    if best_tree is None:
        return TreeNode(prediction=np.mean(y))

    return best_tree

In [ ]:
def predict_one(x, node):
    while node.prediction is None:
        if x[node.feature] <= node.value:
            node = node.left
        else:
            node = node.right
    return node.prediction

def predict(X, tree):
    return np.array([predict_one(x, tree) for x in X])

In [ ]:
df_nump = df.to_numpy()
X_train, y_train, X_test, y_test = df_nump[30:, :-1], df_nump[30:, -1], df_nump[:20, :-1], df_nump[:20, -1]

In [ ]:
gain_std = [0 for i in range(1, 7)]
gain_rmse = [0 for i in range(1, 7)]
for i in range(1, 7):
  tree = build_tree_rss_gain(X_train, y_train, max_depth=i)
  y_pred = predict(X_test, tree)
  if i==1:
      y_gain = y_pred
  print("RSS Gain Tree RMSE:", mean_squared_error(y_test, y_pred)**(1/2))
  gain_rmse[i-1]= round(mean_squared_error(y_test, y_pred)**(1/2),0)
  gain_std[i-1]= round(np.var(y_pred)**(1/2),0)
gain_rmse_std = pd.DataFrame([[gain_rmse[i],gain_std[i]] for i in range(6)])

RSS Gain Tree RMSE: 77532.71410518058
RSS Gain Tree RMSE: 89152.81705235163
RSS Gain Tree RMSE: 86134.21068944888
RSS Gain Tree RMSE: 89279.93553725985
RSS Gain Tree RMSE: 90995.29209630696
RSS Gain Tree RMSE: 89842.88580113254


In [ ]:
local_std = [0 for i in range(1, 7)]
local_rmse = [0 for i in range(1, 7)]
for i in range(1, 7):
  tree = build_tree_rss_local(X_train, y_train, max_depth=i)
  y_pred = predict(X_test, tree)
  if i==1:
      y_local = y_pred
  print("RSS local Tree RMSE:", mean_squared_error(y_test, y_pred)**(1/2))
  local_rmse[i-1]= round(mean_squared_error(y_test, y_pred)**(1/2),0)
  local_std[i-1]= round(np.var(y_pred)**(1/2),0)
local_rmse_std = pd.DataFrame([[local_rmse[i],local_std[i]] for i in range(6)])

RSS local Tree RMSE: 77532.71410518058
RSS local Tree RMSE: 89152.81705235163
RSS local Tree RMSE: 86134.21068944888
RSS local Tree RMSE: 89279.93553725985
RSS local Tree RMSE: 90995.29209630696
RSS local Tree RMSE: 89842.88580113254


In [ ]:
from sklearn.tree import DecisionTreeRegressor
skl_std = [0 for i in range(1, 7)]
skl_rmse = [0 for i in range(1, 7)]
for i in range(1,7):
  drzewo_skl_rss = DecisionTreeRegressor(max_depth=i, random_state=0)
  drzewo_skl_rss.fit(X_train, y_train)
  y_pred =drzewo_skl_rss.predict(X_test)
  print("RSS loc_sklearn Tree RMSE:", mean_squared_error(y_test, y_pred)**(1/2))
  skl_rmse[i-1]= round(mean_squared_error(y_test, y_pred)**(1/2),0)
  skl_std[i-1]= round(np.var(y_pred)**(1/2),0)
skl_rmse_std = pd.DataFrame([[skl_rmse[i],skl_std[i]] for i in range(6)])

RSS loc_sklearn Tree RMSE: 85046.91152603946
RSS loc_sklearn Tree RMSE: 90007.9372718492
RSS loc_sklearn Tree RMSE: 87144.30562416335
RSS loc_sklearn Tree RMSE: 85696.57862165429
RSS loc_sklearn Tree RMSE: 92585.34136489134
RSS loc_sklearn Tree RMSE: 86380.6919576645


In [ ]:
total_std = [0 for i in range(1, 4)]
total_rmse = [0 for i in range(1, 4)]
for i in range(1, 4):
  tree = build_tree_total_rss(X_train, y_train, max_depth=i)
  y_pred = predict(X_test, tree)
  if i==1:
      y_total=y_pred
  print("RSS Total Tree RMSE:", mean_squared_error(y_test, y_pred)**(1/2))
  total_rmse[i-1]= round(mean_squared_error(y_test, y_pred)**(1/2),0)
  total_std[i-1]= round(np.var(y_pred)**(1/2),0)
total_rmse_std = pd.DataFrame([[total_rmse[i],total_std[i]] for i in range(3)])

RSS Total Tree RMSE: 77532.71410518058
RSS Total Tree RMSE: 89152.81705235163
RSS Total Tree RMSE: 84163.69379770114


In [ ]:
wyniki1 = pd.concat([pd.DataFrame(gain_rmse_std), pd.DataFrame(local_rmse_std), pd.DataFrame(skl_rmse_std)], axis=1)
#print(wyniki1.columns)
wyniki1.columns=["rmse_gain","std_gain", "rmse_local","std_local","rmse_skl", "std_skl"]
#print(wyniki1.columns)
wyniki1 = wyniki1.reindex(["rmse_gain","rmse_local","rmse_skl", "std_gain","std_local", "std_skl"], axis=1)
wyniki1.index = [f'max_depth = {i}' for i in range(1,7)]
wyniki1

,rmse_gain,rmse_local,rmse_skl,std_gain,std_local,std_skl
max_depth = 1,77533.0,77533.0,85047.0,69069.0,69069.0,73096.0
max_depth = 2,89153.0,89153.0,90008.0,85783.0,85783.0,82554.0
max_depth = 3,86134.0,86134.0,87144.0,83267.0,83267.0,81092.0
max_depth = 4,89280.0,89280.0,85697.0,84247.0,84247.0,82196.0
max_depth = 5,90995.0,90995.0,92585.0,84959.0,84959.0,81775.0
max_depth = 6,89843.0,89843.0,86381.0,84037.0,84037.0,84281.0


If you find that any pair of formulations is equivalent, provide a concise proof.

Powyższa tabela ukazuje wyniki rmse i odchyleń standardowych dla poszczególnych rodzajów drzew wraz z ustalonymi maksymalnymi ich głębokościami.

 Uwagę przykuwają identyczne wartości tych miar dla drzew zbudownych w oparciu o gain rss i local rss. Rzeczywiście metody te są równoważne ponieważ na każdym etapie gdy maksymalizujemy gain rss obserwacji mieszczących się w węźle rodzica jest ustalone, stąd jedyną zmienną jest suma rss obserwacji z jego dzieci, a zatem zadanie sprowadza się do minimalizacji tej sumy.

 Zauważmy ponadto, że drzewo z domyślnym parametr w dedykowanej funkcji z biblioteki sklearn daje podobne wyniki do pozostałej metody (rmse i odchylenia standardowe różnią się o maksymalnie kilka procent). Nic w tym dziwnego, bo ten parametr to właśnie gain rss, a różnice wynikają między innymi z powodu losowych wyborów podejmowanych w przypadku równości wyników kryterium. Czytelnik może sprawdzić, że wartości miar nieznacznie zmieniają się przy zmianie ustawień losowych.

In [ ]:
wyniki2 = pd.concat([pd.DataFrame(gain_rmse_std).iloc[:3,:], pd.DataFrame(total_rmse_std)], axis=1)
wyniki2.columns=["rmse_gain","std_gain", "rmse_total", "std_total"]
wyniki2= wyniki2.reindex(["rmse_gain", "rmse_total","std_gain", "std_total"], axis=1)
wyniki2.index = [f'max_depth = {i}' for i in range(1,4)]
wyniki2

,rmse_gain,rmse_total,std_gain,std_total
max_depth = 1,77533.0,77533.0,69069.0,69069.0
max_depth = 2,89153.0,89153.0,85783.0,85783.0
max_depth = 3,86134.0,84164.0,83267.0,82976.0


In [ ]:
wyniki3 = [gain_rmse_std.iloc[0,0], local_rmse_std.iloc[0,0], total_rmse_std.iloc[0,0]]
wyniki4 = [gain_rmse_std.iloc[0,1], local_rmse_std.iloc[0,1], total_rmse_std.iloc[0,1]]
print(wyniki3)
print(wyniki4)

[np.float64(77533.0), np.float64(77533.0), np.float64(77533.0)]
[np.float64(69069.0), np.float64(69069.0), np.float64(69069.0)]


If you find that criteria differ, construct a counterexample.

Kryteria gain rss i total rss istotnie się różnią, na co wskazuje 3 rząd powyższej tabelki. Poza samymi budowami drzew różni się także złożoność obliczeniowa. W przypadku total rmse, żeby podjąć decyzje musimy zachłannie rozważyć na wszystkich poziomach, co jest dużo droższe obliczeniowo i powoduje długą kompilacje, która sprawiła że wygenerowałem do ostatecznej analizy jedynie trzy takie drzewa.

Co jednak ważne w przypadku jednego podziału (max_depth =1) metody są równoważne, gdyż total rss sprowadza się do do wyboru na podstawie tylko jednego potencjalnego podziału czyli do local rss.

Compare local RSS minimization, RSS gain maximization, and total RSS minimization.

Jedną z głównych różnic między tymi metodami jest skłonność do przeuczania się modelu local rss ze względu na rozważanie jedynie najlepszego podziału na danym etapie nie zważając czy da to dobre rezultaty przy rozroście drzewa i ucząc się specyfiki danych treningowych niekoniecznie istotnych prawdziwych dla zbioru testowego. Dlatego strategia ta może dawać podobne wyniki dla niewielkic drzew, ale wraz ze wzrostem drzewa metoda total rss okazuje się lepsza. Jednak ze względu na wydajność i dobre metody zapobiegające przeuczeniu (tj. przycinanie drzew) to local rss jest częściej używana.